In [1]:
import sqlite3
import ssl
import urllib.request
import json

url = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"
def download_hsjson(url):
    context = ssl._create_unverified_context()
    req = urllib.request.urlopen(url, context=context)
    f = req.read()
    with open('cards.collectible.json', 'wb') as file:
        file.write(f)
    return f

In [2]:
raw_data = download_hsjson(url)

In [3]:
with open('cards.collectible.json', 'r', encoding='utf-8') as f:
    cards_json = json.load(f)

In [4]:
for i in range(10):
    c = cards_json[i]
    print(c['name'])

Commanding Shout
Museum Curator
Ice Rager
Unearthed Raptor
Fireguard Destroyer
Flametongue Totem
Windfury Harpy
Blade Flurry
Force of Nature
Healing Touch


In [5]:
path_to_db = 'stats.db'

In [6]:
tbl_query = r"SELECT * FROM sqlite_master WHERE type='table'"

create_cards_table_sql = r"""CREATE TABLE "cards" (
	`id`	TEXT NOT NULL,
	`name`	TEXT NOT NULL,
	`rarity`	TEXT NOT NULL,
	`cost`	INTEGER NOT NULL,
	`attack`	INTEGER NOT NULL,
	`health`	INTEGER NOT NULL,
	`set`	TEXT,
	PRIMARY KEY(id)
)"""

create_hero_table_sql = r"""CREATE TABLE "cards" (
	`id`	TEXT NOT NULL,
	`name`	TEXT NOT NULL,
	`rarity`	TEXT NOT NULL,
	`cost`	INTEGER NOT NULL,
	`attack`	INTEGER NOT NULL,
	`health`	INTEGER NOT NULL,
	`set`	TEXT,
	PRIMARY KEY(id)
)"""

create_deck_table_sql = r"""CREATE TABLE `deck` (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`name`	TEXT NOT NULL,
	`tag1`	INTEGER,
	`tag2`	INTEGER
)"""

create_match_table_sql = r"""CREATE TABLE "match" (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`opponent`	INTEGER NOT NULL,
	`first`	INTEGER NOT NULL,
	`won`	INTEGER NOT NULL,
	`duration`	INTEGER NOT NULL,
	`date`	DATETIME NOT NULL,
	`opp_hero`	INTEGER NOT NULL,
	`player_hero`	INTEGER NOT NULL,
	`deck`	INTEGER NOT NULL,
	FOREIGN KEY(`opponent`) REFERENCES `player`(`id`),
	FOREIGN KEY(`opp_hero`) REFERENCES `hero`(`id`),
	FOREIGN KEY(`player_hero`) REFERENCES `hero`(`id`),
	FOREIGN KEY(`deck`) REFERENCES deck(id)
)"""

create_player_table_sql = r"""
CREATE TABLE `player` (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`name`	TEXT NOT NULL,
	`high`	INTEGER NOT NULL,
	`low`	INTEGER NOT NULL
)"""

create_cards_played_sql = r"""CREATE TABLE "card_played" (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`cardid`	INTEGER NOT NULL,
	`turn`	INTEGER NOT NULL,
	FOREIGN KEY(`cardid`) REFERENCES `cards`(`id`)
)"""

insert_card_sql = r"INSERT INTO `cards`(`id`,`name`,`rarity`,`cost`,`attack`,`health`,`set`) VALUES (?,?,?,?,?,?,?);"

In [7]:
db = sqlite3.connect(path_to_db)
db.row_factory = sqlite3.Row
cursor = db.cursor()
result = cursor.execute(tbl_query)
tables = result.fetchall()
db.close()

In [8]:
def check_table_structure(tables, **kw):
    for table in tables:
        name = table['name']
        print(name)

In [9]:
check_table_structure(tables)

cards
player
sqlite_sequence
hero
match
deck
card_played


In [10]:
def update_card_table(cursor, cards):
    for card in cards:
        result = cursor.execute(r"SELECT id from cards WHERE name = ?", (card['name'],))
        row = result.fetchone()
        if row is None:
            # print(card['name'])
            ins = (card['id'], card['name'], card['rarity'], card.get('cost', -1),
                  card.get('attack', -1), card.get('health', -1), card['set'])
            cursor.execute(insert_card_sql, ins)


In [12]:
db = sqlite3.connect(path_to_db)
db.row_factory = sqlite3.Row
cursor = db.cursor()
update_card_table(cursor, cards_json)
db.commit()
db.close()

In [ ]:
for card in cards_json:
    print(card['name'])